### How to Access MERRA-2 Data using OPeNDAP with Python3 and Calculate Daily/Weekly/Monthly Statistics from Hourly Data 

* This instruction is based on Python3 and demonstrates how to remotely access the Modern-Era Retrospective analysis for Research and Applications, Version 2 (MERRA-2) hourly files via OPeNDAP and analyze data such as resample hourly files into daily, weekly, and monthly files and calculate their corresponding statistics, e.g., mean, sum, maximum, and minimum. 

**Contact**: 
* gsfc-dl-help-disc@mail.nasa.gov
* Last update: Sep. 20, 2021

This Python3 example code demonstrates how to remotely access a dataset archived in GES DISC using the Open-source Project for a Network Data Access Protocol (OPeNDAP) web service. We use the Modern-Era Retrospective analysis for Research and Applications, Version 2 (MERRA-2) aerosol diagnostics collection M2T1NXAER.5.12.4 in this example. This collection is 1-hourly time-averaged single-level global aerosol assimilation and archived in daily files with 24 hourly time slices in each file (481 MB per file/day and 14.6 GB per month). For demonstration, we only read 12-day data (i.e., the first 12 days in January 2020)remotely through OPeNDAP URL. We also demonstrate how to calculate the daily/weekly/monthly statistics from hourly data and visualize the evolution of Australian bushfire in January 2020. Figures 1 and 2 are the example images generated by the Python code below, in which the total aerosol extinction (AOT) is plotted as an indicator of the aerosol loading in atmosphere. 

**Prerequisites** 

- This example code is written in Python3 (v3.9.2) Jupyter Notebook and requires these libraries: xarray (0.17.0), matplotlib.pyplot (3.4.1), cartopy.crs (0.18.0), calendar, time, platform (make sure all packages are up to date). In particular, here is the instruction on how to install [xarray](http://xarray.pydata.org/en/stable/getting-started-guide/installing.html) and [cartopy](https://scitools.org.uk/cartopy/docs/v0.15/installing.html). 
- You can execute this example code in your Jupyter Notebook. This code has been tested with Jupyter Notebook v6.2.0 and v6.3.0 in Mac OS, Jupyter Notebook v6.1.4 in Windows OS. Or you can just run it in your Python 3 enviroment. This code has been tested in Python 3 in Mac, window and Linux OS.

**Caveats**:

- Reading multiple hourly data files is a resource demanding task due to large data volume. It may take about 5 minutes to open one-month of the sample data (or longer if the data archive system is  currently heavily loaded). Be patient!
- Visualizing the figures may also take time (~4 minute)
- You may want to test first if your xarray package can read the data in your local disk before reading the data remotely as demonstrated in this case. 


**Reference**:
- [Time-series in xarray](http://xarray.pydata.org/en/stable/time-series.html) 
- [Statistical Operations, Resampling and Climatologies Using Xarray](https://nci-data-training.readthedocs.io/en/latest/_notebook/climate/1_07_Xarray_statistical_resample_roll_climatology_CMIP6.html)
- [Xarray plotting and visualization](https://xarray-contrib.github.io/xarray-tutorial/scipy-tutorial/04_plotting_and_visualization.html)

**Procedure**:
1. Register Earthdata account and set up the credential environment

2. Execute the Python code below in your Jupyter Notebook step-by-step
- 2.1 Import the required Python modules or libraries. If any of the following import commands fail, check the local Python environment and install any missing packages.

In [ ]:
# ----------------------
# Import Python modules
# ----------------------
import warnings
warnings.filterwarnings("ignore")

import xarray as xr, matplotlib.pyplot as plt, cartopy.crs as ccrs, time, platform
from calendar import monthrange

print("platform.python_version() ", platform.python_version())
print(xr.__version__)

In [ ]:
from requests import get
# ADD 'ascii' between url and request-params
# SAMPLE URL 'https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/M2T1NXOCN.5.12.4/2018/09/MERRA2_400.tavg1_2d_ocn_Nx.20180906.nc4.ascii?LWGNTICE[0:1:23][0:1:360][0:1:575],lat[0:1:360],lon[0:1:575],time[0:1:23]'

REQ_FORMAT = 'ascii'
collection_shortname = 'M2T1NXOCN'
collection_longname  = 'tavg1_2d_ocn_Nx'
collection_number = 'MERRA2_400'  
MERRA2_version = '5.12.4'
year = '2018'
month = '9'.zfill(2)
day = '1'.zfill(2)
FEATURE = 'LWGNTICE'
TIME_WINDOWS = '[0:6:23]' # 24 hours divided into 6 hour chunks
LAT_WINDOWS = '[0:1:360]' # 360 deg latitudes
LONG_WINDOWS = '[0:1:575]' # 575 deg longitudes
PARAMS_TO_RECEIVE = [
    FEATURE+TIME_WINDOWS+LAT_WINDOWS+LONG_WINDOWS, 
    'lat'+LAT_WINDOWS, 
    'lon'+LONG_WINDOWS, 
    'time'+TIME_WINDOWS
    ]
tgt_filename = 'temp.txt'


folder_url = f'https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/{collection_shortname}.{MERRA2_version}/{year}/{month}/'
file_url = f'{collection_number}.{collection_longname}.{"".join([year,month,day])}.nc4.{REQ_FORMAT}'
url = folder_url + file_url
url_params = f'{",".join(PARAMS_TO_RECEIVE)}'


response = get(url=url, params=url_params, auth=(''))
try:
    response.raise_for_status()
    with open(tgt_filename, 'wb') as f:
        f.write(response.content)
except Exception as e:
    print('Something went wrong with API GET request.')
    print(e)
finally:
    print('Data Ingestion complete...')

# MAIN CODE FOR DATA PIPELINE

In [6]:
from requests import get
import re, reverse_geocoder as rg, pycountry as pyctry

response = get(
    url='https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/M2T1NXOCN.5.12.4/2018/09/MERRA2_400.tavg1_2d_ocn_Nx.20180906.nc4.ascii', 
    params='TSKINICE[0:4:23][0:5:360][0:5:575],lat[0:5:360],lon[0:5:575],time[0:4:23]'
    )
print(response.status_code)
tgt_filename = r'.\temp.txt'
try:
    response.raise_for_status()
    with open(tgt_filename, 'wb') as f:
        f.write(response.content)
except Exception as e:
    print('Something went wrong with API GET request.')
    print(e)
finally:
    print('Data Ingestion complete...')

200
Data Ingestion complete...


In [7]:
def convert_response_to_json(filename='temp.txt', verbose=False):
    data = None
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.readlines()

    json_data = {}
    for line in data[1:]:
        parts = line.split(',')
        key, values = parts[0], [value.strip() for value in parts[1:]]
        json_data[key] = values
    if verbose:  print(json_data.keys())
    return json_data




def get_country_measurements(locations_vs_measurements, verbose=False):
    if verbose:  print('Creating a {country:(sum,count)} dictionary...')
    country_measurements = dict()
    all_locations = list(locations_vs_measurements.keys())
    all_locations_mapped = rg.search(all_locations)
    for loc, country in zip(all_locations, all_locations_mapped):
        if verbose:  print(loc, country['cc'], locations_vs_measurements[loc])
        if country['cc'] not in country_measurements:
            country_measurements[country['cc']] = [0,0]
        country_measurements[country['cc']][0] += locations_vs_measurements[loc][0]
        country_measurements[country['cc']][1] += 1

    if verbose:  print('Normalizing the measurements taken for each country...')
    total = 0
    for country, (total,counts) in country_measurements.items():
        country_measurements[country] = country_measurements[country][0]/country_measurements[country][1]
        total += country_measurements[country]

    for country, val in country_measurements.items():
        country_measurements[country] = (val/total)*100

    return country_measurements
    
    
    
    
def parse_json_dataset(json_data, verbose=False):
    json_res = dict()
    for t, time_val in enumerate(json_data['time']):
        if verbose:  print(f'Getting measurements across all (latitudes,longitudes) within timestep {time_val}..')
        latitude_rows = []
        for k in json_data.keys():
            if re.findall(f'\[{t}\]\[.*\]', k):
                if verbose:  print(k)
                latitude_rows.append(k)

        locations_vs_measurements = dict()
        for i,latitude_row in enumerate(latitude_rows):
            latitude = int(float(json_data['lat'][i]))
            longitudinal_measurements = json_data[latitude_row]
            for j, measurement in enumerate(longitudinal_measurements):
                longitude = int(float(json_data['lon'][j]))
                location = (latitude,longitude)
                if location not in locations_vs_measurements:
                    locations_vs_measurements[location] = []
                locations_vs_measurements[location].append(100 if measurement=='1e+15' else float(measurement))
        json_res[t] = get_country_measurements(locations_vs_measurements, verbose=False)
        
    return json_res


json_data = convert_response_to_json(filename=tgt_filename)
time_vs_country_measurements = parse_json_dataset(json_data)

In [8]:
import json
output_file = 'temp.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(time_vs_country_measurements, f)

- 2.2 Remotely access the hourly MERRA-2 files through OPeNDAP URL (Please refer to "How to obtain the URL of OPeNDAP served dataset"). In this case below, we read the first 12 days in January 2020. Note that the collection number was changed to a new number if this collection was reprocessed, please refer to ["Records of MERRA-2 Data Reprocessing and Service Changes"](https://disc.gsfc.nasa.gov/information/documents?title=Records%20of%20MERRA-2%20Data%20Reprocessing%20and%20Service%20Changes). The line of "%%time" at the beginning of each cell is used for estimating the running time for that cell. 

In [ ]:
%%time
# ---------------------------------
# Read data
# ---------------------------------
# MERRA-2 collection (hourly)
collection_shortname = 'M2T1NXAER'
collection_longname  = 'tavg1_2d_aer_Nx'
collection_number = 'MERRA2_400'  
MERRA2_version = '5.12.4'
year = 2020
    
# Open dataset
# Read selected days in the same month and year
month = 1  # January
day_beg = 1
day_end = 12
    
# Note that collection_number is MERRA2_401 in a few cases, refer to "Records of MERRA-2 Data Reprocessing and Service Changes"
if year == 2020 and month == 9:
    collection_number = 'MERRA2_401'
            
# OPeNDAP URL 
url = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/{}.{}/{}/{:0>2d}'.format(collection_shortname, MERRA2_version, year, month)
files_month = ['{}/{}.{}.{}{:0>2d}{:0>2d}.nc4'.format(url,collection_number, collection_longname, year, month, days) for days in range(day_beg,day_end+1)]
# Get the number of files
len_files_month=len(files_month)

# Print
print("{} files to be opened:".format(len_files_month))
print("files_month", files_month)

print(" ")  
print("Opening...(It may take ~ 5 minutes to open 1-month data)")
print(" ")
    
# Read dataset URLs
ds = xr.open_mfdataset(files_month)
   
# View metadata (function like ncdump -c)
ds

- 2.3 Select your interested variables and extract it from the whole dataset. In this case, we chose "TOTEXTTAU" (Total Aerosol Extinction) at 550nm as an indicator of the aerosol loading in atmosphere.

In [ ]:
# ---------------------------------------------------
# Select your interested variable (e.g., TOTEXTTAU)
# ---------------------------------------------------
sel_var_shortname = "TOTEXTTAU"
sel_var_value = ds[sel_var_shortname]
sel_var_longname = sel_var_value.attrs['long_name']
sel_var_unit = '('+sel_var_value.attrs['units']+')' 
print("The selected variable is {}: {}{}".format(sel_var_shortname, sel_var_longname,sel_var_unit))

- 2.4 Calculate the daily/weekly/monthly statistics from hourly data

In [ ]:
%%time
# ---------------------------------------------------------------------------------------------------------
# Resample hourly files into daily, weekly, and monthly files and calculate their corresponding statistics,
# e.g., mean, sum, maximum, and minimum.  
# ---------------------------------------------------------------------------------------------------------

# Functions used to calculate various statistics
# ===================================
# Purpose           Function
# 
# mean of dim       mean
# sum of dim        sum
# max of dim        max
# min of dim        min
# ==================================


Get the daily mean

In [ ]:
# Daily mean (i.e., the averaged value over a day at each grid)
sel_var_daily_mean = sel_var_value.resample(time="1D").mean(dim='time', skipna=True)
sel_var_daily_mean

Get the weekly maximum. Note that it is calendar week, starting from Monday to Sunday of each week. Only available data are counted towards each week. For example, for the first week of 2020, it only counts Jan. 1, 2020 (Wed.), which is the first day read in, to Jan. 5, 2020 (Sun.).

In [ ]:
%%time
# Weekly maximum (i.e., the maximum of each week at each grid)
sel_var_weekly_max = sel_var_value.resample(time="1w").max(dim='time', skipna=True)
sel_var_weekly_max

Get monthly total (it doesn't have any physical meaning for this case, AOT. The sum is useful for mass related variable, such as precipitation)

In [ ]:
%%time
# e.g., monthly total (i.e., the total amount over a month at each grid)
# sel_var_monthly_total = sel_var_value.resample(time='m').sum(dim='time', skipna=True)
# sel_var_monthly_total

- 2.5 Visualize the evolution of Australian bushfire with maps and time series

1) Plot the spatial map (e.g., the first two weeks). See Figure 1. 

In [ ]:
%%time
# ------------------------------------------------------------------
# Visualizing the evolution of Australian bushfire 
# ------------------------------------------------------------------

# 1) Plot the spatial facet map (e.g., the first two weeks)
print("Plotting...(It may take ~ 4 minutes)")

pmap = sel_var_weekly_max.isel(time=[0,1]).plot(transform=ccrs.PlateCarree(),  # the data's projection
             col='time', col_wrap=2, robust=True, # multiplot settings
             cmap=plt.cm.Spectral_r, 
             cbar_kwargs={
            "orientation": "horizontal",
            "shrink": 0.9,
            "aspect": 40,
            "pad": 0.1,
                         },
            subplot_kws={'projection': ccrs.PlateCarree(central_longitude=180)})  # the plot's projection
            # shift the original central longitude from 0 to 180 


# We have to set the map's options on all axes

for ax in pmap.axes.flat:
    ax.coastlines(resolution="110m",linewidth=0.5)
    
# Plot main title
main_title = "{}{} weekly_max".format(sel_var_longname, sel_var_unit) 
plt.suptitle(main_title, fontweight='bold')

# Save the plot 
file_dir = '.'
figFile_plot = "{}/map.{}.selected_time.{}.png".format(file_dir, collection_shortname,sel_var_shortname)
plt.savefig(figFile_plot, dpi=200)
print(" Pick up your figure ", figFile_plot) 

2) Plot the time series of daily mean averaged over the globe (Figure 2)

In [ ]:
%%time

# 2) Plot the time series of daily mean averaged over the globe 

fig, ax = plt.subplots(figsize=(16,5.5))
sel_var_daily_mean_region = sel_var_daily_mean.groupby('time').mean(dim=['lat','lon'],skipna=True)

# Convert to dataframe 
plotdata = sel_var_daily_mean_region.to_dataframe()

# List the statistics
stat = plotdata.describe()
print("stat:")
print(stat)

# Plot time series
ax.plot(plotdata,label='daily_mean', marker="o", linewidth=2)
ax.legend(shadow=True, fancybox=True)

# Plot main title and xy labels
main_title = "{}{} daily_mean averaged over the globe".format(sel_var_longname, sel_var_unit) 
plt.suptitle(main_title, fontweight='bold')
ax.set_ylabel(sel_var_shortname+sel_var_unit)

# Save the plot
figFile_plot = "{}/timeseries.{}.selected_time.{}.png".format(file_dir, collection_shortname,sel_var_shortname)
plt.savefig(figFile_plot, dpi=200)
print(" Pick up your figure ", figFile_plot) 